# Training Multilayer Perceptrons

The aim of this tutorial is to learn how to train multilayer perceptrons in Julia. To do so, we will make use of the `Flux` library, whose documentation can be consulted at https://fluxml.ai/Flux.jl/.

If not previously, the packege has to be installed on the system by executing the following commands

In [1]:
using Pkg; Pkg.add("Flux")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


Flux is a library which provides a set of functions to create neural networks with an arbitrary number of layers. This is a library designed to develop Deep Learning projects, whose ANNs usually have a large number of layers of different types, for example, convolutional or maxpooling layers.  In these exercises, only multilayer perceptrons will be developed, with fully-connected (dense) layers, with a maximum of two hidden layers. 

In order to implement an ANN in Julia, there is a function called `Chain`. This function receives as parameters the layers that the network will have (excluding the input layer, which does not perform any processing), which can be of different types.This function receives as parameters the layers that the network will have (excluding the input layer, which does not perform any processing), which can be of different types. Therefore, it is a function with a variable number of parameters.  Depending on the type of layer desired, there are different functions to create each one of them. A couple of examples are the functions `Conv`, which allows for the creation of convolutional layers, or `MaxPool`, which allows for the creation of MaxPooling layers. These layers are used in more advanced models that will be seen in other subjects of the programme and that are beyond the syllabus of this subject.  In this subject, as only multilayer perceptrons will be covered, the layers will always be created fully connected with the function `Dense`. This function accepts as parameters the number of inputs, outputs, and the transfer function of the neurons in the layer. In this sense, two different cases can be distinguished when creating ANNs, depending on the problem to be solved: 

- ***Regression problems***. In this type of problem, the output layer usually has a linear transfer function, while the hidden layers have a non-linear transfer function.  In the following examples, a sigmoidal transfer function is used as the transfer function in the hidden layers; for further information on other supported transfer functions, please refer to the library documentation.  

  In the first example, an ANN is implemented with 10 inputs, a hidden layer with 5 neurons, and an output layer of 1 neuron.

In [2]:
using Flux;

ann = Chain(    
    Dense(10, 5, σ),    
    Dense(5, 1, identity) )

Chain(
  Dense(10 => 5, σ),                    # 55 parameters
  Dense(5 => 1),                        # 6 parameters
)                   # Total: 4 arrays, 61 parameters, 500 bytes.

* The second example builds an ANN with 15 inputs, two hidden layers with 12 and 5 neurons, and an output layer with 2 neurons. 

In [3]:
ann = Chain(    
    Dense(15, 12, σ),    
    Dense(12, 5, σ),    
    Dense(5, 2, identity) )

Chain(
  Dense(15 => 12, σ),                   # 192 parameters
  Dense(12 => 5, σ),                    # 65 parameters
  Dense(5 => 2),                        # 12 parameters
)                   # Total: 6 arrays, 269 parameters, 1.426 KiB.

**Warning:** Be aware that the number of neurons between the different layers has to match

### Question
What would happen if all the layers of the ANN have a linear transfer function?

`If we use a linear transfer function in the hidden layers of the neuron we will make the ANN system to only be able to represent linear functions (i.e. straight lines).`

- ***Classification problems***.  In this case, as explained in theory class, two different situations are considered, depending on whether there are two classes or more than two classes, whereas not belonging to any class is treated as another class:
1. When there are only two classes, this is usually referred to as positives and negatives. In this case, the desired outputs will be either 1 or 0, and a single hidden neuron is, therefore, present.  Thus, such neuron is desired to return values between 0 and 1, which will be interpreted as the degree of certainty the system has that the output is positive.To ensure it returns a bounded value between 0 and 1, a sigmoidal function is applied to the output layer.In the following example, an ANN is defined with a hidden layer and an output neuron; in this example, the sigmoid function has also been used in the hidden layer, but this could be modified.

In [4]:
ann = Chain(    
    Dense(8, 4, σ),    
    Dense(4, 1, σ) )

Chain(
  Dense(8 => 4, σ),                     # 36 parameters
  Dense(4 => 1, σ),                     # 5 parameters
)                   # Total: 4 arrays, 41 parameters, 420 bytes.

2. With more than two classes, you have one output neuron per class. The desired output of a pattern is 1 for the neuron of the class it belongs to, and 0 for the rest. This kind of encoding is known as one-hot-encoding.  In this way, the output of a neuron for a pattern can be interpreted as the degree of certainty that the pattern belongs to the class corresponding to that neuron.  Unlike the previous case, a sigmoidal transfer function is not applied to the outputs of each neuron in the output layer to bound the output between 0 and 1, but no function (identity function) is applied.  Instead, a `softmax` function is applied to the outputs of all the neurons, which takes unbounded numeric values and returns numeric values between 0 and 1 such that the sum of all values is 1.  Even though this function does not constitute a layer of neurons, sometimes this last `softmax` function is considered as an additional layer, and in fact, from the point of view of programming with the `Flux` library, it is effectively performed as if it were a last layer, as can be seen in this example:

In [5]:
ann = Chain(    
    Dense(9, 5, σ),    
    Dense(5, 3, identity),    
    softmax )

Chain(
  Dense(9 => 5, σ),                     # 50 parameters
  Dense(5 => 3),                        # 18 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 68 parameters, 528 bytes.

Another possibility to use the `Chain` function is by successive calls, adding layers to an already created network.  To do this, the ellipsis operator is used when specifying arguments to a function. In the following example, an equivalent ANN is created by first creating an empty ANN and successively adding layers to it: 

In [6]:
ann = Chain(); 
ann = Chain(ann...,  Dense(4, 8, σ) );
ann = Chain(ann...,  Dense(8, 3, identity) ); 
ann = Chain(ann...,  softmax )

Chain(
  Dense(4 => 8, σ),                     # 40 parameters
  Dense(8 => 3),                        # 27 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 67 parameters, 524 bytes.

This variable, obtained in either form, can be used as a function.  For example, the matrix inputs created in the previous tutorial can be taken and passed to the ANN resulting in the outputs of the network by simply writing the following code:

In [7]:
import Pkg; Pkg.add("DelimitedFiles");
using DelimitedFiles 

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [8]:
dataset = readdlm("iris.data",',');

# Split inputs and transform into floats
inputs = dataset[:,1:4]
inputs = convert(Array{Float32,2}, inputs); 

# Split outputs and one hot encode
targets = dataset[:,5]
function encode_categories(targets)
    if (length(unique(targets)) > 2)
        cats = unique(targets) .== permutedims(targets)
        return cats'
    else
        cats = targets .== unique(targets)[1]
        return cats
    end
end

targets = encode_categories(targets)

150×3 adjoint(::BitMatrix) with eltype Bool:
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 ⋮     
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1
 0  0  1

In [9]:
outputs = ann(inputs')

3×150 Matrix{Float32}:
 0.178432  0.179814  0.179357  0.182114  …  0.210366  0.213568  0.211515
 0.558913  0.564351  0.561861  0.564084     0.584702  0.581881  0.588489
 0.262655  0.255835  0.258783  0.253802     0.204932  0.204551  0.199996

***Warning***: due to the formulation used in the world of ANNs, the input and output matrices for the ANN have each pattern in each column, not in each row. Therefore, each row of the input matrix will represent one of the input attributes, while in the output matrix, each row will correspond to one of the desired outputs of the ANN. Consequently, a transpose of the matrices created in the previous practice is required.

Thus, even if the ANN is not properly trained, it may be interesting to make a call similar to this one to verify that the ANN has been created correctly. Once it has been verified that the ANN has been created correctly, it is time to train it. 

To train an ANN, following the workflow described in the theory class, the patterns are presented to the network, then the output is compared with the desired output, and finally a loss value is calculated.  This loss value will be used to modify the weights of the connections and bias. Therefore, a key point is to define this loss function, which will be different for regression and classification problems. The Flux library includes the Losses module with a large number of loss functions used to train NRs. In this subject only the most common ones will be used and, therefore, the first step is to load this module with: 

In [10]:
using Flux.Losses

For any kind of problem, the usage of the `loss` functions is the same. The first argument is the outputs of the model, the second argument is the desired outputs (in both cases with a pattern in each column), and the third argument is the optional keyword `agg`, which indicates how to aggregate the loss values for each pattern. If no value is specified for this keyword, by default an average of all loss values for all patterns will be performed.

* For a regression problem, the most commonly loss function is the **Mean Square Error** (MSE) between the model outputs and the desired targets.  This MSE function is already defined in the Losses module of Flux, although it is very simple to define. It can be used as follows, where `x` are the model inputs and `y` are the targets:

In [11]:
loss(x, y) = Losses.mse(ann(x), y)

loss (generic function with 1 method)

  Other loss functions that may be of interest for regression problems are `Flux.Loss.mae` or `Flux.Loss.msle`.

* For a classification problem the loss function is different. As it was appointed during theory sessions, the binary cross-entropy function is the common choice for a  2-classes problem (only one output neuron).

In [12]:
loss(x, y) = Losses.binarycrossentropy(ann(x), y)

loss (generic function with 1 method)

  Whereas, the cross-entropy function is main used for problems with more than 2  classes (one output neuron per class).

In [13]:
loss(x, y) = Losses.crossentropy(ann(x), y)

loss (generic function with 1 method)

Although, each of these functions could be declared in the various branches of an if statement, Julia has problems when making such function declarations. For this reason, these two declarations can be merged into one, with the following line of code:

In [14]:
loss(x, y) = (size(y,1) == 1) ? Losses.binarycrossentropy(ann(x),y) : Losses.crossentropy(ann(x),y)

loss (generic function with 1 method)

### Question
As it can be seen, first of all the number of rows of the desired output matrix (`y`) is checked, why is the number of rows checked and not the number of columns?

`Because the output matrix y is transposed, making each one of the possible outputs of the classification problem into a row and making each one of the patterns into a column. Meaning, if we want to check if we are in a "binary classification" problem we need to check that the number of rows is 1 (meaning we have either 0 or 1 possible outputs), however in a multiclass classification we need to check if the number of rows is more than one (meaning we can have 001, 010, 100...)`

It is important to remember that in both `x` (inputs) and `y` (targets) each pattern must be in a column, contrary to the usual practice. For this reason, as will be shown below, the matrices of inputs and desired outputs will be transposed.

These functions use the variable `ann`, which is used as a function as it was previously described.  Therefore, it needs to be defined within the environment in which the function is defined.

Once the loss function has been defined, it is necessary to indicate the optimizer to be used during training. The optimizer is nothing more than a specific implementation of one of the alternative backpropagation algorithms. Flux has a large number of those implementations, from the classical one based on gradient descent (`Descent`) or adding also the momentum (`Momentum`) to more advanced ones: `ADAM`, `RADAM`, `AdaMax`, `ADAGrad`, `ADADelta`, `AMSGrad`, `RMSProp`, etc.  Possibly, nowadays, the most widely used optimizer is ADAM, to which you have to indicate the learning rate. This value is usualy a small amount, however, you can find more information in the documentation of the library.

Thus, training **an epoch** of the previous ANN can be done with the `train!` function as follows, where `learningRate` is a variable defined beforehand:

In [15]:
learningRate=2

Flux.train!(loss, Flux.params(ann), [(inputs', targets')], ADAM(learningRate))

"Done"

"Done"

**Important**. It is worth mentioning that in Julia, by convention, when a function is defined by adding `!` (bang) as the last symbol, it is understood taht it modifies the contents of one or more of its arguments, which has therefore been passed by reference.

That is the case of the function `train!`, which has four arguments:

1. The `loss` function, which has been previously defined
2. Weights and bias of the ANN. This can be achieved, as shown in the example, with the function `params`. 
3. A set of patterns, inputs and targets. As you can see in the example, an array with only one element is being passed. This element is a tuple with two elements: the arrays of inputs and desired outputs.  This way of passing the patterns, which may seem cumbersome, has its motivation, since when the set of patterns is very large, calculating the modifications to the weights with all the patterns can be very costly.  For this reason, the patterns are usually divided into batches so that each update is done with only one of these batches. If this were done, the array passed as a parameter, instead of having one tuple, would have several, one per batch.  However, in the exercises to be carried out in this subject, this will not be done, and all the patterns will be passed together. 
  * **Important**: As indicated above, these matrices of inputs and targets have each pattern in a column, contrary to what is usual.  For this reason, the input and target matrices passed as parameters are transposed, i.e. instead of passing `inputs` and `targets`, `inputs'` and `targets'` are passed. If the input and/or target matrices already had a pattern in each column, there would be no need to transpose the corresponding matrix.
  * **Important**: The matrices given in this parameter are used for training the ANN. Therefore, they have to be completely different from those used for testing.
4. Optimizer.  In this example, it is an ADAM with a learning rate equal to  `learningRate`, which usually takes values between 0.001 and 0.1. A common value is 0.01, although you should try different values until you find one that gives good results for the specific problem.

In this way only one loop is trained.  Therefore, to train an ANN it is necessary to create a loop that executes this function as long as some stop criterion is not met. Some of the most common criteria can be:

* The loss in training is good enough.
* The number of training cycles has reached a predefined maximum. 
* The change in training error is less than a predefined value. 
* etc.

#### Question
Could a similar stopping criterion to first one be made but with the test error? Why?

`A similar stop criterium to "the loss in training is good enough" but for test error could lead to overfitting the network and therefore loosing generalization capabilities.`

This way, it is possible to train an ANN so that the error or loss in the training set is minimised.   However, the training of ANNs is not deterministic, but has a random component, which is the random initialisation of the weights.  When this happens, to minimise the random component, the ANN is created and trained several times and the results are averaged.  If you want to train several ANNs, it will be necessary to nest two loops, where the outer loop will iterate through the different networks, and the inner loop will execute the different training cycles of each network.

#### Question
Where in the code (outside both loops, inside the first loop or inside the second loop) will it be necessary to put the call to the Chain function to create each network? Why shouldn't it be put elsewhere?

`It would be inside the first loop in order to initialize the networks with different weights each time, as the second one is devoted to training each network. It cannot be called outside both loops, as that would only create a network once.`

Through this process, the weights and biases, starting from initially random values, will take on different values until one of the stop criteria is met. 

In this sense, it is necessary to bear in mind that the weights corresponding to the connection of inputs with a very high absolute value will take on a low absolute value; on the other hand, the weights of those connections that connect inputs with a low absolute value will take on a high absolute value.  In this way, the ANN is able to combine inputs that are in very different ranges, passing them to values of similar scale.  similar scale values. In other words, the ANN is able to "learn" the relationship between the different scales on which the inputs move.

When dealing with classification problems, the value of loss is often not easy to interpret.  For classification problems there are different metrics that will be the focus of later sessions.  For now, to assess the goodness of output of ANN, we will only use classification accuracy, defined as the ratio of well-classified patterns (number of well-classified patterns divided by the total number of patterns). Therefore, Two different cases can be identified:
1. When there are two classes, the ANN has a single output neuron. As described above, a sigmoidal function, which returns a value between 0 and 1, is typically used as a transfer function. By simply passing a threshold (usually 0.5), the pattern can be classified as "positive" or "negative" depending on whether the output is greater or less than the threshold respectively.
2. When there are more than two classes, applying the softmax function will result in different values of certainty, certainty or probability of belonging to each class.  Therefore, a pattern is classified as the class with the highest probability.

**Important**: This process of normalisation, although it is applied here to ANNs, is common to the rest of the Machine Learning techniques. Therefore, the code to be developed regarding normalisation will also be used in the other models.

On the following paragraph the requirements of the different exercises to devop are explained with the signature of the methods that has to be developed.

1. Develop a function called oneHotEncoding, containing the code developed in the previous session regarding the encoding of a categorical input or output.  That is, to receive a vector of values and encode it as explained in the previous tutorial.  This function will receive two parameters, called `feature` and `classes`, both of type `AbstractArray{<:Any,1}`, that is, they are vectors containing any type of value. The first one has the values of that attribute or desired output for each pattern, and the second one has the values of the categories. This function should perform the following tasks:
  * When the number of values in the class vector is equal to 2, the attribute vector is compared with one of the two classes by broadcasting the `==` operator to generate a vector of Boolean values.   This vector is then transformed into a two-dimensional matrix of one column and returned. To do the latter, see the reshape function.
  * When the number of classes is greater than 2, first a matrix of boolean values is created (of type Array{Bool,2} or BitArray{2}) with as many rows as patterns and as many columns as categories (one column per category). Subsequently, iterate over each column/category, and assign the values of that column as the result of comparing the vector `feature` with the corresponding category by performing a broadcast as in the previous point.

In [16]:
function oneHotEncoding(feature::AbstractArray{<:Any,1}, classes::AbstractArray{<:Any,1})
    numClasses = length(unique(classes))

    if (numClasses == 2)
        oneHot = Array{Bool,2}(undef, size(feature,1), 1)
        oneHot[:,1] .= (feature.==classes[1])
    else
        oneHot = Array{Bool,2}(undef, size(feature,1), numClasses)
        for numClass = 1:numClasses
            oneHot[:,numClass] .= (feature.==classes[numClass])
        end
    end
    return oneHot
end

oneHotEncoding (generic function with 1 method)

2. Overload this function called oneHotEncoding in these two ways: 
  * One that receives a single parameter called `feature`, of type AbstractArray{<:Any,1}, which takes the categories and makes a call to the previous function. The unique function can be used to extract the categories. Develop this function without explicitly declaring it using the word function.

In [17]:
function oneHotEncoding(feature::AbstractArray{<:Any,1})
    return oneHotEncoding(feature, unique(feature))
end

oneHotEncoding (generic function with 2 methods)

  * Another function that receives a single parameter called `feature`, of type AbstractArray{Bool,1}, and therefore it will be a vector that contains for each pattern only two possibilities. As there are two categories, the output array must have a single column, which has the same elements. A simple way to do this function is to use the reshape function.

In [18]:
function oneHotEncoding(feature::AbstractArray{Bool,1})
    return oneHotEncoding(reshape(feature, 1, :))
end

oneHotEncoding (generic function with 3 methods)

3. Develop a set of functions to normalise the data using the code developed in the previous practice.  For this, develop the following functions: 
  * Two functions called `calculateMinMaxNormalizationParameters` and `calculateZeroMeanNormalizationParameters` that receive a parameter of type `AbstractArray{<:Real,2}` and return a tuple with two values, each of them being a matrix with one row with the minimum and maximum values for each column (first function) or means and standard deviations for each column (second function).

In [19]:
using Statistics;

# function defined in Unit1
function stats(outputs)
    minimum = mapslices(Statistics.minimum, outputs; dims=1)[1]
    maximum = mapslices(Statistics.maximum, outputs; dims=1)[1]
    mean = mapslices(Statistics.mean, outputs; dims=1)[1]
    std = mapslices(Statistics.std, outputs; dims=1)[1]
    return [minimum, maximum, mean, std]
end

function calculateMinMaxNormalizationParameters(dataset::AbstractArray{<:Real,2})
    # function that takes a real matrix (i.e. array of reals with dimension 2)
    # this matrix is the data-set to our problem, where each row is a sample and each column is an attribute
    # return a 2-tuple of matrixes where each row is the minimum and maximum respectivelly
    
    min_matrix = []
    max_matrix = []
    
    for column in eachcol(dataset)
        r = stats(column)
        if min_matrix == [] || max_matrix == []
            min_matrix = r[1]
            max_matrix = r[2]
        else
            min_matrix = vcat(min_matrix, r[1])
            max_matrix = hcat(max_matrix, r[2])
        end
    end
    return reshape(min_matrix, (4,1)), reshape(max_matrix, (4,1))
end

calculateMinMaxNormalizationParameters (generic function with 1 method)

In [20]:
function calculateZeroMeanNormalizationParameters(dataset::AbstractArray{<:Real,2})
    means_matrix = []
    std_matrix = []
    
    for column in eachcol(dataset)
        r = stats(column)
        if means_matrix == [] || std_matrix == []
            means_matrix = r[1]
            std_matrix = r[2]
        else
            means_matrix = vcat(means_matrix, r[1])
            std_matrix = vcat(std_matrix, r[2])
        end
    end
    return means_matrix, std_matrix
end

calculateZeroMeanNormalizationParameters (generic function with 1 method)

  * Develop four functions to normalise between maximum and minimum. 
    * The first one, called normalizeMinMax! receives two parameters, an array of values to normalise (of type `AbstractArray{<:Real,2})` and the normalisation parameters (of type `NTuple{2, AbstractArray{<:Real,2}}`), executes the code of the previous tutorial referred to normalise between maximum and minimum, and returns the same array with the normalised data. Subsequently, make another function with the same name (overloaded) but with a single parameter that is the data matrix, and what it will do is to calculate the normalisation parameters with the function developed in the previous point and call the function normalizeMinMax! These two functions end in `!` because the array of values passed as a parameter is modified.

In [21]:
function normalizeMinMax!(dataset::AbstractArray{<:Real,2},      
    normalizationParameters::NTuple{2, AbstractArray{<:Real,2}})
    # x scaled = x - min(x) / max(x) - min(x)
    min, max = normalizationParameters
    for i in axes(dataset, 1)
        for j in axes(dataset, 2)
            cmin, cmax = min[j], max[j]
            dataset[i,j] = dataset[i,j] - cmin / (cmax - cmin)
        end
    end
end

normalizeMinMax! (generic function with 1 method)

In [22]:
function normalizeMinMax!(dataset::AbstractArray{<:Real,2})
    # x scaled = x - min(x) / max(x) - min(x)
    min, max = calculateMinMaxNormalizationParameters(dataset)
    for i in axes(dataset, 1)
        for j in axes(dataset, 2)
            cmin, cmax = min[j], max[j]
            dataset[i,j] = dataset[i,j] - cmin / (cmax - cmin)
        end
    end
end

normalizeMinMax! (generic function with 2 methods)

  Make two other functions with the name normalizeMinMax that do the same, but do not modify the data matrix, i.e. create a new one, modify it and return it.  To do this, see the copy function.  These two functions should make calls to the previous functions.

In [23]:
function normalizeMinMax( dataset::AbstractArray{<:Real,2},      
                normalizationParameters::NTuple{2, AbstractArray{<:Real,2}}) 
    # x scaled = x - min(x) / max(x) - min(x)
    min, max = normalizationParameters
    out = zeros(size(dataset, 1), size(dataset, 2))
    for i in axes(dataset, 1)
        for j in axes(dataset, 2)
            cmin, cmax = min[j], max[j]
            out[i,j] = dataset[i,j] - cmin / (cmax - cmin)
        end
    end
    
    return out
end

normalizeMinMax (generic function with 1 method)

In [24]:
function normalizeMinMax( dataset::AbstractArray{<:Real,2})
    # x scaled = x - min(x) / max(x) - min(x)
    min, max = calculateMinMaxNormalizationParameters(dataset)
    out = zeros(size(dataset, 1), size(dataset, 2))
    for i in axes(dataset, 1)
        for j in axes(dataset, 2)
            cmin, cmax = min[j], max[j]
            out[i,j] = dataset[i,j] - cmin / (cmax - cmin)
        end
    end
    
    return out
end

normalizeMinMax (generic function with 2 methods)

- Develop four similar functions for the case of performing a 0-mean normalisation, whose names are `normalizeZeroMean!` and  `normalizeZeroMean`.

In [25]:
function normalizeZeroMean!(dataset::AbstractArray{<:Real,2},      
                        normalizationParameters::NTuple{2, AbstractArray{<:Real,2}}) 
    mean, std = normalizationParameters
    for i in axes(dataset, 1)
        for j in axes(dataset, 2)
            cmean, cstd = mean[j], std[j]
            dataset[i,j] = dataset[i,j] - cmean / cstd
        end
    end
end
    

normalizeZeroMean! (generic function with 1 method)

In [26]:
function normalizeZeroMean!(dataset::AbstractArray{<:Real,2})
    mean, std = calculateZeroMeanNormalizationParameters(dataset)
    for i in axes(dataset, 1)
        for j in axes(dataset, 2)
            cmean, cstd = mean[j], std[j]
            dataset[i,j] = dataset[i,j] - cmean / cstd
        end
    end
end             

normalizeZeroMean! (generic function with 2 methods)

In [27]:
function normalizeZeroMean( dataset::AbstractArray{<:Real,2},      
                            normalizationParameters::NTuple{2, AbstractArray{<:Real,2}})
    mean, std = normalizationParameters
    out = zeros(size(dataset, 1), size(dataset, 2))
    for i in axes(dataset, 1)
        for j in axes(dataset, 2)
            cmean, cstd = mean[j], std[j]
            out[i,j] = dataset[i,j] - cmean / cstd
        end
    end
    return out
end
                     

normalizeZeroMean (generic function with 1 method)

In [28]:
function normalizeZeroMean( dataset::AbstractArray{<:Real,2}) 
    mean, std = calculateZeroMeanNormalizationParameters(dataset)
    out = zeros(size(dataset, 1), size(dataset, 2))
    for i in axes(dataset, 1)
        for j in axes(dataset, 2)
            cmean, cstd = mean[j], std[j]
            out[i,j] = dataset[i,j] - cmean / cstd
        end
    end
    return out
end        

normalizeZeroMean (generic function with 2 methods)

4. Develop a function called `classifyOutputs`, which receives a parameter called outputs of type `AbstractArray{<:Real,2}`. It will contain the outputs of a model (not necessarily for an ANN) with `a single sample/patten in each row` and convert it to an array of boolean values that each row only has a value of true, indicating the class to which that pattern is classified.  To do this, first look at the number of columns the outputs matrix has, and then do the following: 

   * If you have one column, compare the matrix with a threshold value by broadcasting the >= operator to generate a matrix of boolean values of a column to be returned. To do this, the function needs to receive the threshold value as an optional parameter, with a default value of 0.5. 

   * If you have more than one column, you must create an array of boolean values of the same size, and, for each row, set the column with a larger value to true. This can be done without writing any loops. 

  This aim of this exercise, together with the next one, is to develop skills in  vector programming. Below this lines are the steps that could be taken to write the code for this second scenario, when you have more than one column (assuming that each pattern is in each row):
  
   * First, for each row, it needs to be found out in which column is the maximum output value for each pattern. This can be done with the `findmax` function, which returns a tuple with two values: the maximum in each row or column, and the coordinates in the matrix in which that maximum was found, which is what we are really interested in. With the keyword `dims` you can indicate whether you want to search for the maximums in the rows or in the columns.  The line of code would be as follows:
   ```julia
        (_,indicesMaxEachInstance) = findmax(outputs, dims=2);
    ```
  * Once you have it, you can create a boolean array of the same dimensionality as the output array, where each value indicates the membership of the corresponding class of that pattern. This matrix is initialised to false, so it can be easily created with the function `false`. Such as:
 ```julia
        outputs = falses(size(outputs));
    ``` 
  * Finally, the values of the indices containing the largest values of each row, which are collected in the `indicesMaxEachInstance` variable created earlier, are set to true in this array.  This can be done by comparing with the `outputs` array as follows:
 ```julia
        outputs[indicesMaxEachInstance] .= true;
    ``` 

In [29]:
function classifyOutputs(outputs::AbstractArray{<:Real,2}; 
                        threshold::Real=0.5)
    if size(outputs, 2) == 1
        output = dataset .>= threshold
    else
        (_,indicesMaxEachInstance) = findmax(outputs, dims=2);
        bool_outputs = falses(size(outputs));
        bool_outputs[indicesMaxEachInstance] .= true
    end
    return bool_outputs
end

classifyOutputs (generic function with 1 method)

5.  Develop a function called `accuracy` given a matrix of desired outputs (`targets`) and a matrix of outputs emitted by a model (not necessarily for an ANN), calculate the accuracy in a classification problem.  Both matrices should have a number of rows equal to the number of patterns, i.e. each pattern will be placed in each row. Develop this function in such a way that it works for the cases of having 2 classes (one output neuron) as well as more than two classes (one output neuron per class).

    To develop this function, four functions with the same name must be carried out:
    
    * In the first one, `targets` and `outputs` are of type `AbstractArray{Bool,1}`, i.e. vectors of boolean values.  The precision will simply be the average value of the comparison of both vectors.

In [30]:
using Statistics

function accuracy(outputs::AbstractArray{Bool,1}, targets::AbstractArray{Bool,1}) 
    return Statistics.mean(outputs.==targets)
end

out = [false,true,false,true,false,true]
tar = [true,true,true,true,true,true]

accuracy(out,tar)

0.5

  * In the second of the functions, `targets` and `outputs` are of type `AbstractArray{Bool,2}`, i.e. two-dimensional arrays of Boolean values.  In this case, it will be necessary to examine the number of columns.  If they have only one column, a call to the above function will be made taking the first column of targets and outputs as vectors. If the number of columns is greater than 2, it will be necessary to compare both matrices to see in which rows the values do not match.
  
### Question
 What would happen if the number of columns is equal to 2? (put your answer into a comment in the code below)
 

In [31]:
function accuracy(outputs::AbstractArray{Bool,2}, targets::AbstractArray{Bool,2}) 

    if (size(targets,2)==1)
        return accuracy(outputs[:,1], targets[:,1])
    else
        classComparison = targets .== outputs
        correctClassifications = all(classComparison, dims=2)
        return mean(correctClassifications)
    end
end

out = [false true false true false; 
       false true false true false]
tar = [true true true true true; 
       false true false true false ]

accuracy(out,tar)

0.5

  * In the third of the functions, `targets` will be of type `AbstractArray{Bool,1}` (a vector of boolean values) while `outputs` will be of type `AbstractArray{<:Real,1}`, i.e. real outputs that have not yet been interpreted as "positive"/"negative" class membership values.  In this case, the function could accept an optional `threshold` parameter with a default value of 0.5, and what it would do would be to pass the threshold to the outputs vector, and call the first `accuracy` function.

In [32]:
function accuracy(outputs::AbstractArray{<:Real,1}, targets::AbstractArray{Bool,1}, threshold::Real=0.5)
    return accuracy(Array{Bool,1}(outputs.>=threshold), targets)
end

accuracy (generic function with 4 methods)

  * Finally, in the last of the functions, `targets` will be of type `AbstractArray{Bool,2}` (an array of boolean values) while `outputs` will be of type `AbstractArray{<:Real,2}`, i.e. be real outputs that have not yet been interpreted as values belonging to N classes (N: number of columns). In this case, it is again necessary to distinguish whether you have 1 or more than 2 columns. In the first case, a call to `accuracy` should be made with the vectors corresponding to the first column of outputs and targets. In the second case, a call should be made to the function classifyOutputs to convert outputs into a variable of type AbstractArray{Bool,2}, and then make a call to `accuracy`.

In [33]:
function accuracy(outputs::AbstractArray{<:Real,2}, targets::AbstractArray{Bool,2}, threshold::Real=0.5)
    if (size(targets,2)==1)
        return accuracy(outputs[:,1], targets[:,1])
    else
        classified_outputs=classifyOutputs(outputs)
        return accuracy(classified_outputs, targets)
    end
end

accuracy (generic function with 6 methods)

As in the previous exercise, due to the aim of this one is to develop the vectorial programming skills of the students, the main steps will be pointed out considering that we want ot calculate the accurry of more than 2 classes, the samples are in the rows of the matrices, and both outputs and targgets matrices are `AbstractArray{Bool, 2}`.

  * Once the outputs variable has the desired form (array of Boolean values), it is compared with the targets array as follows: 
   ```julia
        classComparison = targets .== outputs
    ```
  * In this new array, for each pattern, when the class matches, all elements of that row will be true. On the other hand, when the class does not match, more than one element in that row will be false. Therefore, one way to know for a pattern if it is well sorted is to look if in its row all elements are true.  This can be checked with the function `all`, which receives an array and returns true if all elements are true, but also accepts the keyword `dims`, with which it applies this same function across the given dimension. To do this on rows, you would have to do like this:
   ```julia
        correctClassifications = all(classComparison, dims=2)
    ```
  * Finally, the only thing left to do is to average this matrix. Remember that you can operate with boolean values in the same way as with real values, in this case they will be treated as 0 or 1. 
     ```julia
        accuracy = mean(correctClassifications)
    ```
  * These last steps could have been performed by looking, instead of the matches between the two arrays, at the patterns where there are no matches.  For that you can use the function `any`, which receives an array of boolean values and returns true if there is any value equal to true, and also accepts the keyword `dims`. This would calculate the error rate instead of the accuracy, but the accuracy can be calculated from the error rate by simply subtracting it from 1. The code would look like:
      ```julia
        classComparison = targets .!= outputs 
        incorrectClassifications = any(classComparison, dims=2)
        accuracy = 1 - mean(incorrectClassifications)
    ```

6. Develop a function to create ANNs to solve classification problems.  This function must receive the `topology` (number of hidden layers and neurons in each, and optionally activation functions in each hidden layer, the number of input neurons and the number of output neurons).
   **Important** It is worth pointing out that the transfer function of the output layer is not given by the user but by the problem itself (regression/classification).  Similarly, the number of neurons in the input and output layers is given by the problem to be solved.
   A straightforward way to create this ANN is to receive the topology as a parameter called topology of type AbstractArray{<:Int,1}, which contains the number of neurons in each hidden layer (empty for networks without hidden layers), and create the ANN as follows:
   * Create an empty ANN
   ```julia
            ann = Chain();
   ```
   * Create a variable called numInputsLayer, initialized to the number of inputs of the ANN.
   * If there are hidden layers, i.e. if the `topology` vector is not empty, iterate through this vector (the value of the loop will be equal to the number of neurons in each layer) and in each iteration create a hidden layer where the number of inputs will be equal to the value of the numInputsLayer variable and the number of outputs equal to the current value of the loop. If the transfer function of each hidden layer has not been specified, use the same transfer function σ in all hidden layers. After this, update the value of numInputsLayer to the value used in that iteration.
   ```julia
            for numOutputsLayer = topology      
                ann = Chain(ann..., Dense(numInputsLayer, numOutputsLayer, σ) );      
                numInputsLayer = numOutputsLayer; 
            end;
   ```
   If these lines are written in the script without being inside a loop or function, compiling the code will give several warnings and the code will not properly work. This is automatically corrected by using this code inside a function.  To use it in the main body, you should write `global ann, numInputsLayer;` at the beginning of the loop.
   * Finally, add the final layer, with the number of neurons and transfer function appropriate to the number of classes as described above, adding the `softmax` function if there are more than two output classes. 


In [34]:
function buildClassANN(numInputs::Int, topology::AbstractArray{<:Int,1}, numOutputs::Int;
                    transferFunctions::AbstractArray{<:Function,1}=fill(σ, length(topology))) 
    ann = Chain()
    numInputsLayer = numInputs
    for numOutputLayers = topology
        ann = Chain(ann..., Dense(numInputsLayer, numOutputLayers, σ))
        numInputsLayer = numOutputLayers
    end
    if (numOutputs == 1)
        ann = Chain(ann..., Dense(numInputsLayer, 1, σ))
    else
        ann = Chain(ann..., Dense(numInputsLayer, numOutputs, identity))
        ann = Chain(ann..., softmax)
    end
    return ann
end

buildClassANN (generic function with 1 method)

7. Develop a function that creates an ANN to perform classification (via a call to the above function) and trains it.   To do this, this function should implement a loop in which the ANN is trained with the training set passed as a parameter until one of the stop criteria passed as parameters is met.  The function should return the trained ANN.  The argumentes shopuld be:
  * **topology**, of type `AbstractArray{<:Int,1}` with the topology (hidden layers) of the ANN. 
  * **dataset**, of type `Tuple{AbstractArray{<:Real,2},  AbstractArray{Bool,2}} with the  matrix of inputs and targets.  From these matrices, the number of input and output neurons necessary to call the previous function can be obtained by means of the size function.
  * Optional parameters controlling other aspects of the algorithm such as the stopping criteria of the algorithm, with default values:
    * **maxEpochs**, of type Int, with a default value of 1000
    * **minLoss**, of type Real, with a default value of 0
    * **learningRate**, of type Real, with a default value of 0.01.
    
  It is important to highlight that the set of patterns used here will have each pattern in a row, while the Flux library (in general, most Genetic Algorithms libraries do) expect matrices in which each pattern is in a column.  This is not a problem, matrices only need to be transposed on certain occasions, such as when supplying the function to train an ANN cycle, when calculating the loss value, or when taking the output matrices obtained by running the ANN.
  
  Bear in mind that a call to the train! function trains the ANN for one loop only. Therefore, it is necessary, first, to create the ANN by calling the above function, and then to run a loop that calls the `train!` function only once in each iteration.  That is, if you want to train for n cycles, this loop will have to perform `n` iterations.
  
  The output of this function should be at least the trained ANN and a vector with the loss values for each training cycle.

In [35]:
function trainClassANN(topology::AbstractArray{<:Int,1},      
                    dataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,2}};
                    transferFunctions::AbstractArray{<:Function,1}=fill(σ, length(topology)),
                    maxEpochs::Int=1000, minLoss::Real=0.0, learningRate::Real=0.01) 
    
    # Create ANN and loss function for classification problems
    inputs, outputs = dataset
    n_inputs, n_outputs = size(inputs,2), size(outputs,2)
    ann = buildClassANN(n_inputs, topology, n_outputs);
    loss(x, y) = (size(y,1) == 1) ? Losses.binarycrossentropy(ann(x),y) : Losses.crossentropy(ann(x),y)
    
    # Compute base array
    losses = Float64[]
    
    # Compute initial epoch metrics
    current_epoch = 0                    
    current_loss = 999
    # Train
    while current_epoch < maxEpochs || current_loss < minLoss
        Flux.train!(loss, Flux.params(ann), [(inputs', outputs')], ADAM(learningRate))
        
        current_epoch += 1
        ann_outputs = ann(inputs')
        

        current_loss = loss(inputs', ann_outputs);
        acc = accuracy(ann_outputs', Array{Bool,2}(outputs))
        
        push!(losses, current_loss)
        println("epoch:",current_epoch,"; loss:",current_loss,"; accuracy",acc,"...")
    end
    
    return (ann, losses)
end


trainClassANN (generic function with 1 method)

8. Occasionally, when the classification problem is of two classes, instead of having the desired outputs as a single column matrix, it will be as a vector.  To deal with these cases, a function of the same name as above and accepting the same arguments is requested, except that the second argument, dataset, is of type `Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,1}}`. This function should only convert the desired output vector into an array with one column and call the above function.  To do this conversion, see the reshape function.

In [36]:
function trainClassANN(topology::AbstractArray{<:Int,1},      
                    (inputs, targets)::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,1}};      
                    transferFunctions::AbstractArray{<:Function,1}=fill(σ, length(topology)),      
                    maxEpochs::Int=1000, minLoss::Real=0.0, learningRate::Real=0.01)
    targets_reshape = reshape(targets, (size(targets,1),1))
    trainClassANN(topology, (inputs, targets_reshape), transferFunctions, maxEpoch, minLoss, learningRate)
end

trainClassANN (generic function with 2 methods)

In [62]:
norm_input = normalizeMinMax(inputs)

trainClassANN([4,8,3],(norm_input, targets), learningRate=0.5)

epoch:1; loss:0.49098216946321627; accuracy0.3333333333333333...
epoch:2; loss:1.092472140886108; accuracy0.3333333333333333...
epoch:3; loss:1.0079381959100666; accuracy0.3333333333333333...
epoch:4; loss:1.0986122073684819; accuracy0.3333333333333333...
epoch:5; loss:1.017329334455836; accuracy0.3333333333333333...
epoch:6; loss:1.098612173559643; accuracy0.3333333333333333...
epoch:7; loss:1.0173952541709632; accuracy0.3333333333333333...
epoch:8; loss:1.0986120774294628; accuracy0.3333333333333333...
epoch:9; loss:1.0174026847083446; accuracy0.3333333333333333...
epoch:10; loss:1.0986120043576586; accuracy0.3333333333333333...
epoch:11; loss:1.0174089750372397; accuracy0.3333333333333333...
epoch:12; loss:1.0986118183663582; accuracy0.3333333333333333...
epoch:13; loss:1.0174139451170081; accuracy0.3333333333333333...
epoch:14; loss:1.0986115616279704; accuracy0.3333333333333333...
epoch:15; loss:1.028955020463009; accuracy0.3333333333333333...
epoch:16; loss:0.39915812955843666; a

epoch:158; loss:0.5425136968421054; accuracy0.3333333333333333...
epoch:159; loss:1.0197728117536302; accuracy0.3333333333333333...
epoch:160; loss:0.5425787377771015; accuracy0.3333333333333333...
epoch:161; loss:1.019772593904717; accuracy0.3333333333333333...
epoch:162; loss:0.5426439008423418; accuracy0.3333333333333333...
epoch:163; loss:1.0197723703126653; accuracy0.3333333333333333...
epoch:164; loss:0.5427090507979208; accuracy0.3333333333333333...
epoch:165; loss:1.019772156755897; accuracy0.3333333333333333...
epoch:166; loss:0.5427742344808223; accuracy0.3333333333333333...
epoch:167; loss:1.0197719473928817; accuracy0.3333333333333333...
epoch:168; loss:0.542839438605366; accuracy0.3333333333333333...
epoch:169; loss:1.0197717422203716; accuracy0.3333333333333333...
epoch:170; loss:0.5429046959625186; accuracy0.3333333333333333...
epoch:171; loss:1.0197715332701296; accuracy0.3333333333333333...
epoch:172; loss:0.5429700580143582; accuracy0.3333333333333333...
epoch:173; lo

epoch:313; loss:1.0197568001080792; accuracy0.3333333333333333...
epoch:314; loss:0.5477916653821171; accuracy0.3333333333333333...
epoch:315; loss:1.0197566040784811; accuracy0.3333333333333333...
epoch:316; loss:0.5478627735693252; accuracy0.3333333333333333...
epoch:317; loss:1.0197564057379822; accuracy0.3333333333333333...
epoch:318; loss:0.5479340437378621; accuracy0.3333333333333333...
epoch:319; loss:1.019756201924748; accuracy0.3333333333333333...
epoch:320; loss:0.5480054556753178; accuracy0.3333333333333333...
epoch:321; loss:1.019755999706364; accuracy0.3333333333333333...
epoch:322; loss:0.5480769677863021; accuracy0.3333333333333333...
epoch:323; loss:1.0197557998924245; accuracy0.3333333333333333...
epoch:324; loss:0.5481485964144636; accuracy0.3333333333333333...
epoch:325; loss:1.0197556004487456; accuracy0.3333333333333333...
epoch:326; loss:0.5482203208361122; accuracy0.3333333333333333...
epoch:327; loss:1.0197553989415309; accuracy0.3333333333333333...
epoch:328; l

epoch:460; loss:0.5528717140880697; accuracy0.3333333333333333...
epoch:461; loss:1.0197423147906883; accuracy0.3333333333333333...
epoch:462; loss:0.5529304568634007; accuracy0.3333333333333333...
epoch:463; loss:1.0197421317267938; accuracy0.3333333333333333...
epoch:464; loss:0.5529891985979187; accuracy0.3333333333333333...
epoch:465; loss:1.0197419393504235; accuracy0.3333333333333333...
epoch:466; loss:0.5530480492093173; accuracy0.3333333333333333...
epoch:467; loss:1.019741754467334; accuracy0.3333333333333333...
epoch:468; loss:0.5531069475599069; accuracy0.3333333333333333...
epoch:469; loss:1.0197415638588927; accuracy0.3333333333333333...
epoch:470; loss:0.5531659118315623; accuracy0.3333333333333333...
epoch:471; loss:1.019741373183056; accuracy0.3333333333333333...
epoch:472; loss:0.5532249832208079; accuracy0.3333333333333333...
epoch:473; loss:1.0197411844992563; accuracy0.3333333333333333...
epoch:474; loss:0.5532839893120328; accuracy0.3333333333333333...
epoch:475; l

epoch:615; loss:1.019727859916994; accuracy0.3333333333333333...
epoch:616; loss:0.5576362726294514; accuracy0.3333333333333333...
epoch:617; loss:1.0197276691982808; accuracy0.3333333333333333...
epoch:618; loss:0.5577000227428103; accuracy0.3333333333333333...
epoch:619; loss:1.0197274793300244; accuracy0.3333333333333333...
epoch:620; loss:0.5577638663022387; accuracy0.3333333333333333...
epoch:621; loss:1.01972729715175; accuracy0.3333333333333333...
epoch:622; loss:0.5578278048565476; accuracy0.3333333333333333...
epoch:623; loss:1.0197271125797673; accuracy0.3333333333333333...
epoch:624; loss:0.5578919339432679; accuracy0.3333333333333333...
epoch:625; loss:1.0197269224801566; accuracy0.3333333333333333...
epoch:626; loss:0.5579561025434632; accuracy0.3333333333333333...
epoch:627; loss:1.0197267361646736; accuracy0.3333333333333333...
epoch:628; loss:0.5580203420858509; accuracy0.3333333333333333...
epoch:629; loss:1.0197265482466833; accuracy0.3333333333333333...
epoch:630; lo

epoch:771; loss:1.0197135146775247; accuracy0.3333333333333333...
epoch:772; loss:0.5628750333314317; accuracy0.3333333333333333...
epoch:773; loss:1.0197133340445423; accuracy0.3333333333333333...
epoch:774; loss:0.5629461319631806; accuracy0.3333333333333333...
epoch:775; loss:1.0197131518026408; accuracy0.3333333333333333...
epoch:776; loss:0.5630173715889047; accuracy0.3333333333333333...
epoch:777; loss:1.0197129719401858; accuracy0.3333333333333333...
epoch:778; loss:0.563088667902127; accuracy0.3333333333333333...
epoch:779; loss:1.0197127919259967; accuracy0.3333333333333333...
epoch:780; loss:0.5631601453346654; accuracy0.3333333333333333...
epoch:781; loss:1.0197126157499043; accuracy0.3333333333333333...
epoch:782; loss:0.5632317632082585; accuracy0.3333333333333333...
epoch:783; loss:1.0197124299025857; accuracy0.3333333333333333...
epoch:784; loss:0.563303477337819; accuracy0.3333333333333333...
epoch:785; loss:1.0197122494783035; accuracy0.3333333333333333...
epoch:786; l

epoch:927; loss:1.019699461339929; accuracy0.3333333333333333...
epoch:928; loss:0.5688320544442472; accuracy0.3333333333333333...
epoch:929; loss:1.0196992842204582; accuracy0.3333333333333333...
epoch:930; loss:0.5689148271336454; accuracy0.3333333333333333...
epoch:931; loss:1.0196991007303533; accuracy0.3333333333333333...
epoch:932; loss:0.5689978150942675; accuracy0.3333333333333333...
epoch:933; loss:1.0196989234070415; accuracy0.3333333333333333...
epoch:934; loss:0.569080908509155; accuracy0.3333333333333333...
epoch:935; loss:1.0196987422870605; accuracy0.3333333333333333...
epoch:936; loss:0.569164299795281; accuracy0.3333333333333333...
epoch:937; loss:1.019698560865062; accuracy0.3333333333333333...
epoch:938; loss:0.5692479187194062; accuracy0.3333333333333333...
epoch:939; loss:1.019698379740821; accuracy0.3333333333333333...
epoch:940; loss:0.5693316655432629; accuracy0.3333333333333333...
epoch:941; loss:1.0196982048457506; accuracy0.3333333333333333...
epoch:942; loss

(Chain(Dense(4 => 4, σ), Dense(4 => 8, σ), Dense(8 => 3, σ), Dense(3 => 3), softmax), [0.49098216946321627, 1.092472140886108, 1.0079381959100666, 1.0986122073684819, 1.017329334455836, 1.098612173559643, 1.0173952541709632, 1.0986120774294628, 1.0174026847083446, 1.0986120043576586  …  1.0196937581480774, 0.5715846015340789, 1.019693579778409, 0.5716743463082123, 1.0196933999530917, 0.5717642514367276, 1.019693217918197, 0.5718544847308444, 1.0196930413263487, 0.5719450058610056])

#### Warning
Remember to integrate this functions with the code developed on the previous tutorial in a separate file.

Once the training function returns a trained ANN, it can be used to simulated on different problems by passing it a set of inputs (with the patterns in columns, i.e. transposed) and will return the outputs for that set (again, the patterns will be in columns, so the outputs of the ANN will have to be transposed).  These outputs, along with the desired outputs for that set of inputs, can be applied to the accuracy function to calculate the accuracy on that set. 

Run multiple times to train different networks with different architectures - which one will give the highest accuracy on the training set? Also test different learning rate values.

`If we use a big topology like [3,9,27,48,65] the network performs badly, obtaining an average of 0.6 accuracy. If we use a small topology like [1] the network also performs badly, obtaing an average of 0.7 accuracy. When using an topology shaped like the input and output data like [4,8,3] we obtain average better results with an 0.97 accuracy. For the best topology ([4,8,3]) we tried different learning rates: for 0.01 the results flucutate between 0.97 and 0.98, for a greater LR of 0.1 the results fluctuate between 0.65 and 0.7. For greater LR values like 0.5 the results were the worst with a 0.33 accuracy.` 

Repeat the experiments performed above, with the unstandardised data, and compare the results with the standardised data. Are there important differences when using standardised data? Which one is the est topology?

`The experiments were performed using MinMax normalization technique. When repeating the experiments without the normalization applied the results were overall lower, ranking with lower accuracy metrics for almost all topologies.

Overall, we can say that the best topology is the one that better fits the given problem [4,8,3] and the best LR is the lowest one. We can also say that the normalisation techniques provide useful increases in metrics`

# Julia Notes

As any other language, Julia allows the creation of functions. There are several ways to create functions, the most common are these two:

1. When the operation to be performed is simple, the function can be declared on one line without the need for the reserved word function.  This is the case for operations that can be performed in one or a few lines of code.  In the latter case, parentheses can be used to enclose the actions to be performed, and `;` to separate them.  The value to be returned by the function will be the last thing to be evaluated, although the reserved word `return` can also be used.  Here are a couple of examples of this way of declaring functions:
    ```julia
         add(x::Float32, y::Float32) = x+x; 
         mse(outputs::Array{Float32,1}, targets:: Array{Float32,1}) = mean((targets.-outputs).^2);
         avgGreaterThan0(valores::Array{Float32,1}) = mean(values[values.>0]); 
         avgGreaterThan0(valores::Array{Float32,1}) = ( positives=values.>0; mean(values[positives]); )   
    ```
2. In many other cases, a function may perform many complex operations that are impractical to write on a single line.  In this case, you can declare the function with the reserved word `function`, and return the result with the reserved word `return`.  As is common in programming languages, evaluating `return` immediately exits the function.  If `return` is not used, the result returned by function will be the last one evaluated.  The example above is shown below: 
   ```julia
         function avgGreaterThan0(Values::Array{Float32,1})    
            positives=values.>0;    
            return mean(Values[positives]); 
         end;  
    ```
  When passing parameters, it is not mandatory, but recommended, to indicate the type of the parameters.  If this is not done, they are assumed to be of type `Any`.  However, a common practice in Julia is to overload functions, i.e. to define functions with the same name but different parameters or of different types.  When calling a function, the correct function will be executed, if any of the ones defined match the parameters passed. This allows different behaviours to be defined. When a function call is made, Julia checks that there exists in memory a definition with that name in which the types of the passed arguments match the defined ones and, if it exists, it will execute it with those parameters.
  
  In the examples above, the parameters have been defined as `Float32` or `Array{Float32,2}`, which means that the functions are defined for those specific types, and therefore, if calls are made with parameters of type `Float64` or `Array{Float64,2}`, a suitable function definition will not be found, and an error will be raised.  To solve this, it is worthy to apply the subtyping properties as seen in the previous tutorial to define the most generic types that the function can work with. For example, instead of using `Float32`, you can use `AbstractFloat`, `Real` or `Number` types (be aware that the latter type includes complex numbers). Instead of using `Array{Float32,2}`, you could use `Array{<:AbstractFloat}`, `Array{<:Real}` or `Array{<:Number}`. Additionaly, it is needed to take into remember that there are other types that behave like arrays but are not arrays. An important example that will be used in this subject is when transposing arrays, which creates an object that is not of type `Array`, but of type `LinearAlgebra.Adjoint`, but that can be used as if it were an array because its operations are defined as such.  Both `Array` and `LinearAlgebra.Adjoint` are subtypes of `AbstractArray`, as can be seen in the following example: 

In [38]:
m = [1. 2.; 3. 4.]; 

In [39]:
isa(m, Array)

true

In [40]:
isa(m, AbstractArray)

true

In [41]:
typeof(m') 

LinearAlgebra.Adjoint{Float64, Matrix{Float64}}

In [42]:
isa(m', Array)

false

In [43]:
isa(m', AbstractArray)

true

Since both types, `Array` and `LinearAlgebra.Adjoint` are subtypes of `AbstractArray`, to allow an argument to be of one type or the other, this will be the type to be used in the function arguments. Therefore, the above functions will be as follows: 

   ```julia
        add(x::Real, y:: Real) = x+x;
        mse(outputs::AbstractArray{<:Real,1},targets::AbstractArray{<:Real,1})=mean((targets.-outputs).^2)
        avgGreaterThan0(values::AbstractArray{<:Real,1}) = mean(values[values.>0]); 
        avgGreaterThan0(values:: AbstractArray{<:Real,1})=(positives=values.>0; mean(values[poitives]);) 
        
        function avgGreaterThan0(values::AbstractArray{<:Real,1})    
            positives=values.>0;    
            return mean(Values[positives]); 
        end;  
   ```
When an argument is a vector or array of boolean values, as is the case of the desired outputs in a classification problem, the type to use will be `AbstractArray{Bool,N}`, where N is the dimensionality of the array.  As mentioned in the previous tutorial, this type is a supertype of both `Array{Bool,N}` and `BitArray{N}`. 

An interesting feature of Julia functions is that they can return more than one value. This is done by using the type `Tuple{...}`, which designates tuples where each element has a certain type, for example `Tuple{Float32,Float32}`, `Tuple{Array{Float32,2}, Int64}` or `Tuple{Float32, Tuple{Int64, Int4}}`. In general, if you want to return more than one element, you return a tuple with the elements you want to return, for example:

   ```julia
        function avgGreaterThan0(values::AbstractArray{<:Real,1})    
            positives=valores.>0;    
            return (positives, mean(values[positives])) 
        end; 

        (positives, average) = avgGreaterThan0([1.2 , -1.3 , 5.5 , -3.8 , -2.1])
   ```

When creating tuples where all elements have the same type, a simplified way to define the type is to use `NTuple`, specifying the number of elements and the type. For example:

In [44]:
Tuple{Float64,Float64}==NTuple{2,Float64}

true

As was aforementioned, the names of those functions that modify the values of one of their arguments are usually terminated with `!` by convention.